In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Example directory structure (adapt as needed):
# data/
#   train/
#       healthy/
#           image1.jpg
#           image2.jpg
#           ...
#       disease1/
#           image1.jpg
#           ...
#       disease2/
#           ...
#   val/
#       healthy/
#       disease1/
#       disease2/
# etc.

data_dir = "data"  # Adjust this path to your dataset directory
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")

# Image transformations: resize, random augmentations, normalize
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),               # or 256x256
    transforms.RandomHorizontalFlip(p=0.5),      # data augmentation
    transforms.RandomRotation(degrees=15),       # data augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])  # Typical ImageNet normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

# Create Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(root=val_dir,   transform=val_transforms)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False, num_workers=2)

# Number of classes (e.g., 'healthy', 'disease1', 'disease2', ...)
num_classes = len(train_dataset.classes)
print("Classes detected:", train_dataset.classes)


NameError: name 'os' is not defined

In [ ]:
class PlantDiseaseCNN(nn.Module):
    def __init__(self, num_classes):
        super(PlantDiseaseCNN, self).__init__()
        
        # Convolutional block 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1   = nn.BatchNorm2d(32)
        self.pool  = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Convolutional block 2
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(64)
        
        # Convolutional block 3
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3   = nn.BatchNorm2d(128)
        
        # Fully connected layers
        self.fc1   = nn.Linear(128 * 28 * 28, 256)  # after 3 conv layers + 2 pools => check shape
        self.fc2   = nn.Linear(256, 128)
        self.fc3   = nn.Linear(128, num_classes)
        
        self.relu  = nn.ReLU()

    def forward(self, x):
        # Block 1: Conv -> BN -> ReLU -> Pool
        x = self.conv1(x)               # Z^{(1)}
        x = self.bn1(x)
        x = self.relu(x)                # A^{(1)}
        x = self.pool(x)                # P^{(1)} reduces H,W by 2

        # Block 2: Conv -> BN -> ReLU -> Pool
        x = self.conv2(x)               # Z^{(2)}
        x = self.bn2(x)
        x = self.relu(x)                # A^{(2)}
        x = self.pool(x)                # P^{(2)}

        # Block 3: Conv -> BN -> ReLU -> Pool (optional)
        x = self.conv3(x)               # Z^{(3)}
        x = self.bn3(x)
        x = self.relu(x)                # A^{(3)}
        # We can optionally pool again if desired:
        # x = self.pool(x)  # uncomment if you'd like to reduce further

        # Flatten
        x = x.view(x.size(0), -1)       # vector \mathbf{h}

        # Fully connected layers
        x = self.fc1(x)                 # z1
        x = self.relu(x)
        x = self.fc2(x)                 # z2
        x = self.relu(x)
        x = self.fc3(x)                 # z3 (logits)

        return x

model = PlantDiseaseCNN(num_classes=num_classes)
print(model)


In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()  # Cross-entropy:  -\sum y_i \log(\hat{y}_i)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # or SGD, etc.


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device='cuda'):
    model = model.to(device)
    
    for epoch in range(num_epochs):
        model.train()  # set to training mode
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)            # logits
            loss = criterion(outputs, labels)  # cross-entropy loss

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        epoch_loss = running_loss / total
        epoch_acc = 100.0 * correct / total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for val_images, val_labels in val_loader:
                val_images = val_images.to(device)
                val_labels = val_labels.to(device)
                
                val_outputs = model(val_images)
                v_loss = criterion(val_outputs, val_labels)
                
                val_loss += v_loss.item() * val_images.size(0)
                _, val_predicted = torch.max(val_outputs, 1)
                val_correct += (val_predicted == val_labels).sum().item()
                val_total += val_labels.size(0)
        
        val_epoch_loss = val_loss / val_total
        val_epoch_acc = 100.0 * val_correct / val_total
        
        print(f"Epoch [{epoch+1}/{num_epochs}] "
              f"Train Loss: {epoch_loss:.4f} Train Acc: {epoch_acc:.2f}% "
              f"Val Loss: {val_epoch_loss:.4f} Val Acc: {val_epoch_acc:.2f}%")

    return model

# Train for 10 epochs (example)
device = "cuda" if torch.cuda.is_available() else "cpu"
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device=device)


In [ ]:
import torchvision.transforms.functional as TF
from PIL import Image

def predict_single_image(model, image_path, device='cuda'):
    # Switch to eval mode
    model.eval()
    
    # Load and preprocess the image
    img = Image.open(image_path).convert('RGB')
    transform_pipe = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
    tensor_img = transform_pipe(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        outputs = model(tensor_img)
        _, predicted_class = torch.max(outputs, 1)
    
    class_idx = predicted_class.item()
    print(f"Predicted class index: {class_idx} -- {train_dataset.classes[class_idx]}")
    return class_idx

# Example usage:
# predict_single_image(trained_model, "path/to/test_image.jpg", device=device)
